In [1]:
import cv2
import numpy as np
import imageio as io
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize,StandardScaler
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
import skimage.io as io
from skimage.feature import local_binary_pattern
from skimage import color
from operator import eq
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd

In [2]:
def extract_hog_features(img):
    img = img.astype(int)
    target_img_size = (78, 78)
    img = cv2.resize(img.astype('uint8'), target_img_size)
    #64--.i.e 2 block
    win_size = (64, 64)
    cell_size = (8, 8)
    block_size_in_cells = (2, 2)

    block_size = (block_size_in_cells[1] * cell_size[1],
                  block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 15  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size,
                            block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()

    return h.flatten()

In [3]:
def preproccessing(img):
    gray=color.rgb2gray(img)
    return gray

In [4]:
# path_to_dataset =r'D:\Download\jaffedbase'
path_to_dataset =r'E:\emotion\archive\ck\CK+48'
# path_to_dataset =r'E:\datasetemotion\val\val'
img_filenames = os.listdir(path_to_dataset)

In [5]:
img_filenames = os.listdir(path_to_dataset)
print(img_filenames)

['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']


In [6]:
random_seed = 0  
random.seed(random_seed)
np.random.seed(random_seed)

In [7]:
def load_dataset(feature_set='hog'):
    
    features = []
    labels = []

    for i, fn in enumerate(img_filenames):
            label = fn.split('.')[0]
            path = os.path.join(path_to_dataset, fn)
            subpath=os.listdir(path)
            for k, d in enumerate(subpath):             
                imgpath = os.path.join(path, d)
                img = io.imread(imgpath)
                img = cv2.resize(img,(48,48), interpolation = cv2.INTER_AREA)
                img=color.rgb2gray(img)
                if(np.array(extract_features(img)).any()!=None):
                    labels.append(label)
                    features.append(extract_features(img))
                if k > 0 and k % 100 == 0:
                    print("[INFO] processed {}/{}".format(k, len(subpath)))
    return features, labels            

In [8]:
def extract_features(img):
    return extract_hog_features(img) 

In [9]:
def split_data(features,labels,precentage):
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=random_seed)
    return train_features, test_features, train_labels, test_labels

In [10]:
def KNN_classifier(kcount,train_features, train_labels):
    classifier = KNeighborsClassifier(n_neighbors=kcount)
    classifier.fit(list(train_features),list(train_labels) )
    return classifier

In [11]:
def normalization(train_features, test_features):
    scaler = StandardScaler()
    scaler.fit(train_features)
    train_features = scaler.transform(train_features)
    test_features = scaler.transform(test_features)
    return train_features, test_features

In [12]:
def test_classifier(test_features, test_labels,classifier):
        accuracy = classifier.score(test_features, test_labels)        
        print('accuracy:', accuracy*100, '%')

In [13]:
print('Loading dataset. This will take time ...')
features, labels = load_dataset(feature_set='hog')

print('Finished loading dataset.')

Loading dataset. This will take time ...
[INFO] processed 100/135
[INFO] processed 100/177
[INFO] processed 100/207
[INFO] processed 200/207
[INFO] processed 100/249
[INFO] processed 200/249
Finished loading dataset.


In [14]:
features1, labels1 =features, labels 

In [15]:
print(np.array(features1).shape)
print(np.array(labels1).shape)

(981, 11760)
(981,)


In [16]:
train_features, test_features, train_labels, test_labels=split_data(np.array(features1),np.array(labels1),0.2)

In [17]:
from sklearn.metrics.pairwise import rbf_kernel
le = preprocessing.LabelEncoder()
random_seed = 0  
random.seed(random_seed)
np.random.seed(random_seed)
sigma=0.0008
c=64
reg=100
err=1e-4
#Get Kernel Function
def kernel(X, Y=None,g=None):
    k=rbf_kernel(X, Y, gamma=g)
    return k
#Get Output Weight "Fitting Classifier"
def KELM_Beta(sigma,c,x,y):
    le.fit(y)
    y = le.transform(y)
    emotion = len(np.unique(y))
    One_Hot_Encoding=np.zeros((emotion,emotion), np.uint8)
    for em in range (emotion):
        One_Hot_Encoding[em][em]=1
    T = One_Hot_Encoding[y, :]    
    N,d=x.shape
    Omega=kernel(x,None,sigma)
    I=np.eye(N)
    Beta = np.linalg.inv((I /c) + Omega).dot(T)    
    return Beta,One_Hot_Encoding
#Get Predicted Labels "Predication"
def KELM_Output(sigma,x,x_test,Beta,One_Hot_Encoding):
    hx=kernel(x_test,x,sigma)
    fx= hx.dot(Beta)
    p=np.argmax(fx,axis=1)
    return p
def acc(y_pred,test_labels):
    le = preprocessing.LabelEncoder()
    yt = le.fit_transform(y_pred)    
    rt = le.fit_transform(test_labels)
    eqw = sum(map(eq, list(rt), list(y_pred)))
    size=len(rt)
    acc=(eqw/size)*100
    return acc
Beta,One_Hot_Encoding=KELM_Beta(sigma,c,np.array(train_features),np.array(train_labels))
p1=KELM_Output(sigma,np.array(train_features),np.array(test_features),Beta,One_Hot_Encoding)
print("test",acc(p1,test_labels))

p2=KELM_Output(sigma,np.array(train_features),np.array(train_features),Beta,One_Hot_Encoding)
print("train",acc(p2,train_labels))

test 100.0
train 100.0


In [18]:
#KNN

In [19]:
# print(train_features)
print(np.array(train_features).shape)
print(np.array(train_labels).shape)

(784, 11760)
(784,)


In [20]:
classifier=KNN_classifier(3,train_features, train_labels)


In [21]:
y_pred = classifier.predict(test_features)

In [22]:
def acc(y_pred,test_labels):
    le = preprocessing.LabelEncoder()
    yt = le.fit_transform(y_pred)
    rt = le.fit_transform(test_labels)
    eqw = sum(map(eq, list(rt), list(yt)))
    size=len(rt)
    acc=(eqw/size)*100
    print(acc)
    print(yt)
    print(rt)
    
    return acc


In [23]:
accuracy = classifier.score(test_features, test_labels)
print('Test accuracy:', accuracy*100, '%')
accuracy = classifier.score(train_features, train_labels)
print('Train accuracy:', accuracy*100, '%')

Test accuracy: 77.66497461928934 %
Train accuracy: 97.32142857142857 %


In [24]:
print(accuracy_score(test_labels, y_pred))
print(classification_report(test_labels, y_pred))
print(confusion_matrix(test_labels, y_pred))

0.7766497461928934
              precision    recall  f1-score   support

       anger       0.75      0.90      0.82        20
    contempt       0.75      1.00      0.86         6
     disgust       0.87      0.66      0.75        41
        fear       0.68      0.87      0.76        15
       happy       0.82      0.85      0.84        39
     sadness       0.55      0.55      0.55        22
    surprise       0.83      0.81      0.82        54

    accuracy                           0.78       197
   macro avg       0.75      0.80      0.77       197
weighted avg       0.78      0.78      0.78       197

[[18  0  0  0  0  2  0]
 [ 0  6  0  0  0  0  0]
 [ 4  0 27  2  5  2  1]
 [ 0  0  0 13  0  2  0]
 [ 2  0  2  0 33  0  2]
 [ 0  2  2  0  0 12  6]
 [ 0  0  0  4  2  4 44]]


In [25]:
import joblib
# joblib.dump(classifier, 'DellKNN2.pkl')


In [103]:
# img  = io.imread('KA.AN2.40.tiff')
img  = io.imread('MK.HA1.116.tiff')
# img  = io.imread('UY.DI2.150.tiff')



f=extract_features(img)
print(f)

[0.18610714 0.21370839 0.15028238 ... 0.21837309 0.09984857 0.10134899]


In [104]:
y_pred = classifier.predict([f] )
print(y_pred)

['Happy']


In [105]:
#SVM

In [26]:
from sklearn.svm import SVC
# k in ['rbf','sigmoid']\
# gamma in ['scale','auto']

k='sigmoid'
gamma = 'auto'
c=400
clf = SVC(C=c,kernel=k,gamma=gamma)
clf.fit(train_features, train_labels)
print ('train score', clf.score(train_features, train_labels)*100 )
y=clf.predict(test_features)
print ('test score', clf.score(test_features,test_labels)*100 )

train score 100.0
test score 100.0


In [27]:
print(accuracy_score(test_labels, y))
print(classification_report(test_labels, y))
print(confusion_matrix(test_labels, y))

1.0
              precision    recall  f1-score   support

       anger       1.00      1.00      1.00        20
    contempt       1.00      1.00      1.00         6
     disgust       1.00      1.00      1.00        41
        fear       1.00      1.00      1.00        15
       happy       1.00      1.00      1.00        39
     sadness       1.00      1.00      1.00        22
    surprise       1.00      1.00      1.00        54

    accuracy                           1.00       197
   macro avg       1.00      1.00      1.00       197
weighted avg       1.00      1.00      1.00       197

[[20  0  0  0  0  0  0]
 [ 0  6  0  0  0  0  0]
 [ 0  0 41  0  0  0  0]
 [ 0  0  0 15  0  0  0]
 [ 0  0  0  0 39  0  0]
 [ 0  0  0  0  0 22  0]
 [ 0  0  0  0  0  0 54]]


In [32]:
# img  = io.imread('KA.AN2.40.tiff')
img  = io.imread('UY.DI2.150.tiff')
# img  = io.imread('KL.SU1.164.tiff')
# img  = io.imread('S010_004_00000017.png')
# img  = io.imread('MK.HA1.116.tiff')
f=extract_features(img)
print(f)

In [54]:
y_predsvm=clf.predict([f])
print(y_predsvm)

['happy']


In [83]:
import joblib
# joblib.dump(clf, 'DellSVM.pkl')

['DellSVM.pkl']